In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.manual_seed(999)

In [2]:
working_dir = '/media/jswaney/Drive/MNIST'

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(os.path.join(working_dir, 'data'),
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.1307,), std=(0.3081,))])),
    batch_size=64,
    shuffle=True)

In [5]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(os.path.join(working_dir, 'data'),
                   train=False,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.1307,), std=(0.3081,))])),
    batch_size=64,
    shuffle=True)

In [20]:
class MyNetwork(nn.Module):
    
    def __init__(self):
        super().__init__()
        # Create the layers to be used in the model
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  # Defaults to 50% dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        # Define the model architecture
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)  # Flatten feature maps
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)  # Module class must have a "training" flag
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)  # log softmax is probably easier to optimize

In [21]:
model = MyNetwork().to(device)

In [22]:
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [23]:
def training_step(model, device, train_loader, optimizer, epoch):
    model.train()  # sets training mode / flag in Module class
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # reset the parameter gradients
        output = model(data)
        loss = F.nll_loss(output, target)  # negative log-likelihood added to model
        loss.backward()  # compute gradients for the whole model
        optimizer.step()  # update model parameters with current gradients
    print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch, loss.item()))

In [24]:
def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1] 
            correct += pred.eq(target.view_as(pred)).sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
for epoch in range(1, 5):
    training_step(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

RuntimeError: invalid argument 2: size '[-1 x 321]' is invalid for input with 20480 elements at /opt/conda/conda-bld/pytorch_1525812548180/work/aten/src/TH/THStorage.c:37